In [ ]:
#!pip freeze

### 第一次参加比赛落下帷幕，使用的是FasterRCNN50，图像增强使用的是某位大佬提供的notebook，不算伪代码的话训练了50epoch左右，但是从最后的结果来看，哪怕再增加训练的轮数也已经不能够得到更好的LB了。
### 观看其他大佬的notebook和discussion，需要学习的有几点：
1. 额外数据的寻找和导入
2. 混合模型的使用（虽然并不是很懂这一个概念）或者说应该尝试更多的方法，当然这一次是因为加入之后剩余的时间本来就不多了
3. EDA的执行（这次并没有很好的观察数据与图像本身）
4. 整体框架copy，整合了其他的图像增强的方式，但是其中部分方法并没有很好的理解，需要后续仔细。另外应当深入到图像增强的源代码当中  
下面先将notebook中不懂的部分标出，留待后续解决
不过好歹也算往入门更近了一步，另外，非常感谢[Jacob](https://www.kaggle.com/creatrol)大佬 [gzYou](https://www.kaggle.com/guizengyou)大佬不厌其烦的回复和解疑，虽然问的问题有一点点(to be honest a lot)愚蠢。
非常喜欢这一个氛围，希望能够完成更多的比赛学习到更多新的东西。

After Wheat competition, I'd like to record something incluing the problems and the inspirations.  
model：FasterRCNN50(after COCO pretrain)  
epoch:50
external way:Mosaic and Pseudo label
The question is in the main body and the inspiration is below:  
1. Search and import of additional data
2. The use of the hybrid model (although not very familiar with this concept) or should try more methods, of course, this time because there is not much time left after joining
3. EDA implementation (there is no good observation data and image itself this time)
4. The whole framework copy integrates other image enhancement methods, but some of them are not well understood and need to be followed up carefully. In addition, we should go deep into the source code of image enhancement

Now I'll mark the part that I don't understand in the notebook, and I'll leave it for later solution. But somehow, it's a step closer to the beginning. Thank you very much for your untiring reply and answering questions, [Jacob](https://www.kaggle.com/creatrol) and [gzYou](https://www.kaggle.com/guizengyou)，although the question I asked is a little bit stupid.   
I like Kaggle’s atmosphere very much. I hope I can finish more competitions and learn more new things.


In [ ]:
import glob
from tqdm import notebook

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt

DIR_INPUT = '../input/global-wheat-detection'
DIR_TRAIN = f'{DIR_INPUT}/train'
DIR_TEST = f'{DIR_INPUT}/test'

DIR_WEIGHTS = '../input/lasttry/fasterrcnn_resnet50_fpn12nd.pth'

WEIGHTS_FILE = f'{DIR_WEIGHTS}' 

In [ ]:
test_df = pd.read_csv(f'{DIR_INPUT}/sample_submission.csv')
#test_df

In [ ]:
class WheatTestDataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        #self.image_ids = dataframe['image_id'].unique()
        self.image_ids=np.array([path.split('/')[-1][:-4]for path in glob.glob(f'{image_dir}/*.jpg')])
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        #records = self.df[self.df['image_id'] == image_id]

        image = cv2.imread(f'{self.image_dir}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        if self.transforms:
            sample = {
                'image': image,
            }
            sample = self.transforms(**sample)
            image = sample['image']

        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

1.从目录中读取图片名称而不是从csv中读取，这样就能够对接在submit时的隐藏test，避免出现提交错误  
2.cv库读取的图片格式在大部分情况下都需要使用RGB转换，相对于其他图片读取库而言，cv的速度更快速，这使得它在深度学习大规模图片读取中应用非常广泛，当然，存在需要转换RGB格式这一个无关紧要的缺陷  
3.对于我们这样的初学者而言，需要注意的反而是图像导入转化过程中格式的变化，如pandas、numpy、tensor几个数据格式之间的转换，当然还有tuple，根据我浅薄的见识，tuple因为不能更改更加安全

1. Read the image name from the directory instead of from the CSV. In this way, you can dock the hidden test in submit and avoid submission errors
2. In most cases, the image format read by CV library needs to use RGB conversion. Compared with other image reading libraries, the speed of CV is faster, which makes it widely used in large-scale image reading of deep learning. Of course, there is an insignificant defect of converting RGB format
3. For beginners like us, what we need to pay attention to is the change of format in the process of image import and transformation, such as the conversion between pandas, numpy, tensor, and of course, tuple. According to my shallow knowledge, tuple is safer because it can't be changed

In [ ]:
# Albumentations
def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


~~目前并没有很好的使用的数据增强方法，使用的手段是非常简易的翻转~~
后来还是使用了数据增强，但是也是copy的，虽然也花了我不少时间去调试格式的问题，其实想做的是将所有方式全部使用AL库的compose进行整合，毕竟其提供了加入自定义函数的方法，不过目前有点力所不逮，
但是我看一个notebook讲masico的方式循环十轮左右就满足了，我似乎训练的有点多了。
下面是从AL文档中copy过来的参数解释：
transforms (list) – list of transformations to compose.  
bbox_params (BboxParams) – Parameters for bounding boxes transforms

~~At present, there is no good data enhancement method, and the method used is very simple flip~~  
Later, data enhancement was used, but it was also copy. Although it took me a lot of time to debug the format problem, what I wanted to do was to integrate all the methods using the compose of Al library.   
After all, it provided a way to add custom functions, but at present, it was a little difficult,But I see a notebook to talk about masico, and it's enough to cycle about ten rounds.   
I seem to have trained a little bit more.
The following is an explanation of the parameters copied from the Al document:  
transforms (list) – list of transformations to compose.  
bbox_ params (BboxParams) – Parameters for bounding boxes transforms

In [ ]:
# Albumentations
def get_test_transform():
    return A.Compose([
        # A.Resize(512, 512),
        ToTensorV2(p=1.0)
    ])


In [ ]:
# load a model; pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)

The introduction of residual network pre trained by coco and training on this basis should be regarded as migration training.   
However, I have no good understanding of the principle of migration training

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 2  # 1 class (wheat) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load the trained weights
model.load_state_dict(torch.load(WEIGHTS_FILE))
model.eval()

x = model.to(device)

虽然这一步就是简单的模型导入，但是不得不承认，对于in_feature的导入这一句我还是有点疑问的，后面还得抽空去看Pytorch的文档。

Although this step is a simple model import, I have to admit that for in_ I still have some doubts about the import of feature.  
I have to take time to read the document of Pytorch.

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

test_dataset = WheatTestDataset(test_df, DIR_TEST, get_test_transform())

test_data_loader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

其实def collate_fn(batch):这一个语句我还是有点没有理解，虽然我去查看了DataLoader的参数

Actually, def collate_ FN (batch): I still don't understand this statement, although I went to check the parameters of dataloader

In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

# # **Detection and make Pseudo labels for test dataset**

In [ ]:
detection_threshold = 0.5
results = []

testdf_psuedo = []
for images, image_ids in test_data_loader:

    images = list(image.to(device) for image in images)
    outputs = model(images)

    for i, image in enumerate(images):

        boxes = outputs[i]['boxes'].data.cpu().numpy()
        scores = outputs[i]['scores'].data.cpu().numpy()
        
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        scores = scores[scores >= detection_threshold]
        image_id = image_ids[i]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
        for box in boxes:
            #print(box)
            result = {
                'image_id': 'nvnn'+image_id,
                'width': 1024,
                'height': 1024,
                'source': 'nvnn',
                'x': box[0],
                'y': box[1],
                'w': box[2],
                'h': box[3]
            }
            testdf_psuedo.append(result)
            


其实我这里很想尝试一下，不同的detection_threshold会带来的变化，因此先码在这里。
另一个就是开始的时候我还有疑问就是为什么这里的数据还要导入到cpu，后来才知道要运算的话只能转换，这大概就是基础不扎实吧。

In fact, I'd like to try different detection methods here_ Threshold will bring changes, so code here first.  
The other is that at the beginning, I still had a question about why the data here has to be imported into the CPU.   
Later, I learned that if you want to calculate, you can only convert it. This is probably because my foundation is not solid.

In [ ]:
test_df_pseudo = pd.DataFrame(testdf_psuedo, columns=['image_id', 'width', 'height', 'source', 'x', 'y', 'w', 'h'])
test_df_pseudo.head()

# # **Retrain model with pseudo labels

In [ ]:
train_df = pd.read_csv(f'{DIR_INPUT}/train.csv')
train_df['x'] = -1
train_df['y'] = -1
train_df['w'] = -1
train_df['h'] = -1

def expand_bbox(x):
    r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x))
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return r

train_df[['x', 'y', 'w', 'h']] = np.stack(train_df['bbox'].apply(lambda x: expand_bbox(x)))
train_df.drop(columns=['bbox'], inplace=True)
train_df['x'] = train_df['x'].astype(np.float)
train_df['y'] = train_df['y'].astype(np.float)
train_df['w'] = train_df['w'].astype(np.float)
train_df['h'] = train_df['h'].astype(np.float)

train_df.head()

正则的语句和方法真是看了忘看了忘，只好copy了

The regular statements and methods are really forgotten, so I have to copy them

In [ ]:
image_ids = train_df['image_id'].unique()
valid_ids = image_ids[-665:]
train_ids = image_ids #[:-665]

valid_df = train_df[train_df['image_id'].isin(valid_ids)]
#train_df = train_df[train_df['image_id'].isin(train_ids)]

In [ ]:
frames = [train_df, test_df_pseudo]

train_df = pd.concat(frames)
train_df.tail()

# # **Wheat dataset for Training**

In [ ]:
class WheatDataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        records = self.df[self.df['image_id'] == image_id]
        if 'nvnn' in image_id:
            image_id = image_id[4:]
            image = cv2.imread(f'{DIR_TEST}/{image_id}.jpg', cv2.IMREAD_COLOR)
        else:
            image = cv2.imread(f'{self.image_dir}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)

        # there is only one class
        labels = torch.ones((records.shape[0],), dtype=torch.int64)
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        # target['masks'] = None
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]
    

说实话，这个数据导入函数我已经翻来覆去看来很多遍了，但是总觉得还差点意思，比如target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)这一句，，这个几重转换都要把我搞晕了，，不知道有没有大佬可以解答一下
还有就是这个面积计算我总感觉有那么一点怪

To tell you the truth, I've looked over and over for this data import function many times, but I always think it's almost meaningless,  
such as target ['boxes']= torch.stack (tuple(map( torch.tensor , zip (* sample ['bboxes']))). Permute (1, 0)   
This several conversions will make me confused. I wonder if there is a big guy who can solve it.  
There is also the area calculation, I always feel a little strange,is it necessary？

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0
        
        
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = WheatDataset(train_df, DIR_TRAIN, get_train_transform())
valid_dataset = WheatDataset(valid_df, DIR_TRAIN, get_valid_transform())


# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# **Show a sample**

In [ ]:
images, targets, image_ids = next(iter(train_data_loader))
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

boxes = targets[2]['boxes'].cpu().numpy().astype(np.int32)
sample = images[2].permute(1,2,0).cpu().numpy()

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

这里有一个小小的疑问就是在数据导入时已经做了均一化，即/255了，这里并没有做任何其他操作便进行display了，是不是cv本身自带的对应的补全方法

There is a small question here, that is, / 255 has been homogenized when data is imported. Display is performed without any other operation.  
Is it the corresponding completion method of CV itself？

# # ****retrain Faster RCNN****

In [ ]:
model.train()
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, weight_decay=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
#lr_scheduler = None

num_epochs =30

loss_hist = Averager()
itr = 1

for epoch in range(num_epochs):
    loss_hist.reset()
    
    for images, targets, image_ids in train_data_loader :
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.long().to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}")


加tqdm包，，这样的话训练的过程更赏心悦目

In fact，When I run the code in my own computer,I add tqdm which I think let the running time more interesting.

In [ ]:
torch.save(model.state_dict(), 'fasterrcnn_resnet50_fpn3nd.pth')

对于参数保存的版本下次一定得做好记录，，我几次训练之后，，就乱套了。

I need to pay attention to the records of parameters version which confused me this time.

# **final Inference and submission

In [ ]:
model.eval()
detection_threshold=0.2
results=[]
for images, image_ids in test_data_loader:

    images = list(image.to(device) for image in images)
    outputs = model(images)

    for i, image in enumerate(images):

        boxes = outputs[i]['boxes'].data.cpu().numpy()
        scores = outputs[i]['scores'].data.cpu().numpy()
        
        boxes = boxes[scores >= detection_threshold].astype(np.int32).clip(min=0,max=1023)
        scores = scores[scores >= detection_threshold]
        image_id = image_ids[i]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
        
            
        result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)
        }

        
        results.append(result)


In [ ]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.head()

In [ ]:
sample = images[1].permute(1,2,0).cpu().numpy()
boxes = outputs[1]['boxes'].data.cpu().numpy()
scores = outputs[1]['scores'].data.cpu().numpy()

boxes = boxes[scores >= detection_threshold].astype(np.int32)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 2)
    
ax.set_axis_off()
ax.imshow(sample)

In [ ]:
test_df.to_csv('submission.csv', index=False)